In [33]:
import os
import sys

import requests
import json
from bs4 import BeautifulSoup
from urllib.request import urlretrieve

import numpy as np
import pandas as pd
from tqdm import tqdm

import cv2
from PIL import Image

In [90]:
def getRepThumb(category):
    
    url = f'https://comic.naver.com/webtoon/genre.nhn?genre={category}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    items = soup.select('div.list_area > ul > li')
    
    category_endpoints = []
    for item in items:
        endpoint = 'https://comic.naver.com'
        web_url = endpoint + item.select_one('div.thumb > a')['href']
        web_title = item.select_one('div.thumb > a')['title']
        web_thumb = item.select_one('div.thumb > a > img')['src']
        
        DATA_PATH = 'dataset'
        if not os.path.exists(DATA_PATH):
            os.mkdir(DATA_PATH)
        
        FILE_PATH = os.path.join(DATA_PATH, category)
        if not os.path.exists(FILE_PATH):
            os.makedirs(FILE_PATH)
        
        FILE_NAME = FILE_PATH + f'/{web_title}.jpg'
        urlretrieve(web_thumb, FILE_NAME)
        category_endpoints.append([category, web_title, web_url])
        
    return category_endpoints

In [58]:
url = 'https://comic.naver.com/webtoon/genre.nhn?genre=episode'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
categories = soup.select('div.snb > ul.spot > li')
categories = [elem.text.strip() for elem in categories]

In [91]:
web_urls = []
for category in tqdm(categories):
    category_urls = getRepThumb(category)
    web_urls.extend(category_urls)

100%|██████████| 13/13 [00:03<00:00,  3.53it/s]


In [92]:
webtoon = pd.DataFrame(web_urls, columns = ['category', 'title', 'url'])

In [95]:
webtoon

,category,title,url
0,에피소드,독립일기,https://comic.naver.com/webtoon/list.nhn?title...
1,에피소드,모죠의 일지,https://comic.naver.com/webtoon/list.nhn?title...
2,에피소드,한림체육관,https://comic.naver.com/webtoon/list.nhn?title...
3,에피소드,세상은 돈과 권력,https://comic.naver.com/webtoon/list.nhn?title...
4,에피소드,지옥급식,https://comic.naver.com/webtoon/list.nhn?title...
...,...,...,...
2270,스포츠,가족의 초상,https://comic.naver.com/webtoon/list.nhn?title...
2271,스포츠,동경소녀,https://comic.naver.com/webtoon/list.nhn?title...
2272,스포츠,한스와 에밀리,https://comic.naver.com/webtoon/list.nhn?title...
2273,스포츠,징글정글,https://comic.naver.com/webtoon/list.nhn?title...


In [123]:
def getdetailThumbs(url):
    thumb_list = []
    for i in range(1, 3):
        try:
            url = url + f"&page={i}"
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            items = soup.select('tr > td > a > img')
        
        
            for item in items:
                if item['alt'] == 'AD 배너':
                    continue

                thumb_list.append(item['src'])
        except:
            pass
            
    return thumb_list

In [125]:
for category, title, url in tqdm(webtoon.values):
    
    thumb_list = getdetailThumbs(url)
    
    for i, thumb in enumerate(thumb_list, start = 1):
        FILE_NAME = os.path.join('dataset', category, f'{title}_{i}.jpg')
        urlretrieve(thumb, FILE_NAME)

100%|██████████| 2275/2275 [55:37<00:00,  1.47s/it] 


In [127]:
file_num = 0
categories = os.listdir('./dataset')

for category in categories:
    FILE_PATH = os.path.join('dataset', category)
    file_num += len(os.listdir(FILE_PATH))

In [128]:
print(file_num)

42030
